In [ ]:
def lagged_features(df
    ,target_var = ['aces_match_player1']
    , lags = [1]
    , windows = [28]
    , aggregation_methods = {'mean', 'median', percentile(10), percentile(90)}
    , surface_type_filter = [1, -1]):
    ## with no surface type filter
    players = ['player1', 'player2']
    df = df.reset_index()
    for p in players:
        if -1 in surface_type_filter:
            for l, w, t in product(lags, windows, target_var):
                t1 = t
                if p == 'player2':
                    t1 = t1.replace("player1", "player2")
                lf_df = df.set_index(['player1', 'player2', 'date'])[t1].\
                         groupby(level=p).apply(lambda x: x.rolling(window=w, min_periods = 1).agg(aggregation_methods).shift(l))
                t1 = t1.replace("player1", "").replace("player2", "")
                new_names = {x: "{0}_lag_{1}_{2}_{3}_{4}".
                              format(p.replace("player", "p"), l, t1, x, w) for x in lf_df.columns}

                df = df.merge(lf_df.reset_index().rename(columns = new_names),
                    how='left', on =['player1', 'player2', 'date'] )
        ## with surface type filter
        if 1 in surface_type_filter:
            for l, w, t in product(lags, windows, target_var):
                t1 = t
                if p == 'player2':
                    t1 = t1.replace("player1", "player2")
                df.head()
                lf_df = df.set_index(['player1', 'player2', 'date', 'Surface'])[t1].\
                         groupby(level=[p, 'Surface']).apply(lambda x: x.rolling(window=w, min_periods = 1).agg(aggregation_methods).shift(l))
                t1 = t1.replace("player1", "").replace("player2", "")
                new_names = {x: "{0}_filter_lag_{1}_{2}_{3}_{4}".
                              format(p.replace("player", "p"), l, t1, x, w) for x in lf_df.columns}

                df = df.merge(lf_df.reset_index().rename(columns = new_names),
                    how='left', on =['player1', 'player2', 'date', 'Surface'])
    return df
